In [102]:
import numpy as np
import pandas as pd
from scipy.stats import truncnorm

In [103]:
def gen_list_names(df):
    ## Implied that every team plays at least 1 home game in the data set
    return df["team1"].unique()

def gen_dict(name_list):
    return dict.fromkeys(name_list, (25, pow(25/3, 2)))

In [104]:
def init_data():
    df = pd.read_csv("SerieA.csv")
    # df.info()
    no_ties_df = df.loc[df["score1"] != df["score2"]]
    # no_ties_df.info()
    init_dict = gen_dict(gen_list_names(df))
    return no_ties_df, init_dict

# Random variables
### s1 ~ N(m_s1, sigma_s1^2)
### s2 ~ N(m_s2, sigma_s2^2)
### t = N(s1 - s2, sigma_t²2)
### y = sign(t)

# Factors
### factor_s1 = N(s1; m_s1, sigma_s1^2)
### factor_s2 = N(s2; m_s2, sigma_s2^2)
### factor_s1s2t(s1,s2,t) = N(t; s1-s2, sigma_t^2)
### factor_ty(t, y) = dirac(y= sign(t))

# Gaussian cheat sheet methods
## Copied from ex. 7.1

In [105]:
def multiplyGauss(m1,s1,m2,s2):
    # computes the Gaussian dist N(m,s) proportional to N(m1,s1)*N(m2,s2)
    s = 1/(1/s1 + 1/s2)
    m = (m1/s1 + m2/s2)*s
    return m,s


In [106]:
def divideGauss(m1, s1, m2, s2):
    # computes the Gaussian dist N(m,s) proportional to N(m1,s1)/N(m2,s2)
    return multiplyGauss(m1,s1,m2,-s2)

In [107]:
def truncGaussMM(a,b, m0, s0):
    # a and b are scaled intervals with the mean and variance, see moment_matching()
    return truncnorm.mean(a, b, loc=m0, scale=np.sqrt(s0)), truncnorm.var(a, b, loc=m0, scale=np.sqrt(s0))

In [108]:
def moment_matching(y, message_m, message_s):
    if y == 1:
        a, b = (0 - message_m)/np.sqrt(message_s), np.inf
    else:
        a, b = -np.inf, (0 - message_m)/np.sqrt(message_s)
    return truncGaussMM(a, b, message_m, message_s)


# Helper methods from earlier exercise

In [109]:
def m_n(Sn, S0, m0, var, X, y):
    return Sn @ ( np.add(S0.I @ m0, 1/var * X.T * y) )

def s_n(S0, var, X):
    return ( np.add(S0.I, 1/var * X.T @ X) ).I

In [110]:
# Message passing algorithm
def message_pass(df, team_dict):
    for i, row in df.iterrows():
        
        if row['score1'] > row['score2']:
            y = 1
        else:
            y = -1

        team1_prior_mu = team_dict[row['team1']][0]
        team1_prior_s = team_dict[row['team1']][1]
        team2_prior_mu = team_dict[row['team2']][0]
        team2_prior_s = team_dict[row['team2']][1]

        # print('*' * 10)
        # print("prior dist s1 s2")
        # print(row['team1'], team1_prior_mu, team1_prior_s)
        # print(row['team2'], team2_prior_mu, team2_prior_s)

        ## Message from S1 -> fs1s2t
        mu1_m = team1_prior_mu
        mu1_s = team1_prior_s

        ## Message from S2 -> fs1s2t
        mu2_m = team2_prior_mu
        mu2_s = team2_prior_s

        ## Message from fs1s2t -> t = N(mu1_m - mu2_m, mu1_s + mu_2s)
        mu3_m = mu1_m - mu2_m
        mu3_s = mu1_s + mu2_s

        ## Moment matching on y, since fty results in a truncated norm, approximate the message with a gaussian distribution
        pt_m, pt_s = moment_matching(y, mu3_m, mu3_s)
        
        # print('*'*10)
        # print("moment matching pt_m, pt_s", pt_m, pt_s)

        ## Compute the message from t|y to f_s1s2t by dividing q(t)/mu_3(t)
        mu4_m, mu4_s = divideGauss(pt_m, pt_s, mu3_m, mu3_s)
        
        ## Define X, m0 and s0 from our model
        X = np.array([[1, -1]])
        m0 = np.array([[mu1_m],
                       [mu2_m]])
        S0 = np.matrix([[mu1_s, 0],
                        [0, mu2_s]])
        
        ## Compute the marginals of s1, s2 (the message from f_s1s2t to s1,s2)
        Sn = s_n(S0, mu4_s, X)
        mn = m_n(Sn, S0, m0, mu4_s, X, mu4_m)

        ps1_m, ps1_s = mn[0], Sn[0,0]
        ps2_m, ps2_s = mn[1], Sn[1,1]
        
        # print('*'*10)
        # print("posterior dist s1 s2")
        # print(row['team1'], ps1_m[0,0], ps1_s)
        # print(row['team2'], ps2_m[0,0], ps2_s)
        
        ## Update s1 and s2
        team_dict[row['team1']] = ps1_m[0,0], ps1_s
        team_dict[row['team2']] = ps2_m[0,0], ps2_s
        
    return team_dict


In [111]:
def gen_df_from_dict(dictionary):
    results_pd = pd.DataFrame.from_dict(dictionary, orient='index', columns=["mean", "var"])
    return results_pd



In [112]:
def __main():
    df, teams_dict = init_data()
    updated_dict = message_pass(df, teams_dict)
    # print(updated_dict)
    updated_df = gen_df_from_dict(updated_dict)
    return updated_df


In [113]:
updated_df = __main()
updated_df.sort_values('mean')

,mean,var
Fiorentina,24.674793,0.031060
Genoa,24.863935,0.060198
Parma,24.987264,0.044288
Frosinone,25.045617,0.023410
Sassuolo,25.051749,0.086008
Juventus,25.131428,0.015859
Sampdoria,25.248904,0.013500
Chievo,25.284497,0.040599
Cagliari,25.347280,0.009456
Spal,25.408057,0.009876
